# Read burden analysis (R code)

# Install packages

In [1]:
# Utility routine for installing packages
install_if_missing <- function(packages) {
    if (length(setdiff(packages, rownames(installed.packages()))) > 0) {
        install.packages(setdiff(packages, rownames(installed.packages())))
    }
}
install_if_missing(c('tidyverse', 'data.table'))
print('Done!!')

Installing package into ‘/home/jupyter-user/notebooks/packages’
(as ‘lib’ is unspecified)



[1] "Done!!"


# Load libs

In [1]:
library(tidyverse)
library(data.table)
print('Done!!')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose




[1] "Done!!"


# Utility functions

In [2]:
# Utility routine for printing a shell command before executing it
shell_do <- function(command) {
    print(paste('Executing: ', command))
    system(command, intern = TRUE)
}

# Utility routines for reading files from Google BigQuery
bq_query <- function(query) {
    # Return the contents of a query against BigQuery    
    return(bigrquery::bq_table_download(
        bigrquery::bq_project_query(BILLING_PROJECT_ID, query = query)))
}

# Initialize authorization for BigQuery operations
bigrquery::bq_auth(path=Ronaldo::getServiceAccountKey())

print('Done !!')

[1] "Done !!"


# Global

In [3]:
BILLING_PROJECT_ID <- Sys.getenv('GOOGLE_PROJECT')
WORKSPACE_NAMESPACE <- Sys.getenv('WORKSPACE_NAMESPACE')
WORKSPACE_NAME <- Sys.getenv('WORKSPACE_NAME')
WORKSPACE_BUCKET <- Sys.getenv('WORKSPACE_BUCKET')

print(str_glue('BILLING_PROJECT_ID = {BILLING_PROJECT_ID}'))
print(str_glue('WORKSPACE_NAMESPACE = {WORKSPACE_NAMESPACE}'))
print(str_glue('WORKSPACE_NAME = {WORKSPACE_NAME}'))
print(str_glue('WORKSPACE_BUCKET = {WORKSPACE_BUCKET}'))

googleGSId='fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5'

print('Done !!')

BILLING_PROJECT_ID = larobak11
WORKSPACE_NAMESPACE = larobak11
WORKSPACE_NAME = LSDProject
WORKSPACE_BUCKET = gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5
[1] "Done !!"


# Copy from bucket to instance

In [14]:
#copy files from bucket to instance
#gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5/notebooks/LSDGenes_51_genes_paperSupplement.txt
#fileName='LSDGenes_51_genes_paperSupplement.txt'
fileName='plink_pheno_withPCs_rvtestFormat_allRaces.tab'
cmd = str_glue('gsutil cp gs://{googleGSId}/notebooks/{fileName} {dataDir}')
cat('cmd:', cmd, '\n')
system(cmd)
print('Done!')

cmd: gsutil cp gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5/notebooks/plink_pheno_withPCs_rvtestFormat_allRaces.tab /home/jupyter-user/notebooks/LSDProject/data/ 
[1] "Done!"


# Global paths

In [4]:
#dataDir='../LSD-All-genes'
dataDir='/home/jupyter-user/notebooks/LSDProject/data/'
resDir='/home/jupyter-user/notebooks/LSDProject/results/LSDGenes-WGS_Dec3120/'
print(str_glue('dataDir:{dataDir}'))
print(str_glue('resDir:{resDir}'))
print(str_glue('current dir:{getwd()}'))
plinkCmd='/home/jupyter-user/notebooks/LSDProject/tools/plink'
#cmd=str_glue('ls -lh {dataDir}')
#shell_do(cmd)
print('Done')

dataDir:/home/jupyter-user/notebooks/LSDProject/data/
resDir:/home/jupyter-user/notebooks/LSDProject/results/LSDGenes-WGS_Dec3120/
current dir:/home/jupyter-user/notebooks/LSDProject/edit
[1] "Done"


# Read Annovar

In [5]:
annoPrefix='LSD-GENES'
annoFile=str_glue('{resDir}{annoPrefix}.annovar.hg38_multianno.txt')
print(str_glue('annoFile: {annoFile}'))
#no need below, already done in part 1 script
#fileName=annoFile
annoDf = fread(annoFile,  header=T, sep='\t',stringsAsFactors = F, check.names=F)
cat('dim:', dim(annoDf), '\n')
print('Done !!')

annoFile: /home/jupyter-user/notebooks/LSDProject/results/LSDGenes-WGS_Dec3120/LSD-GENES.annovar.hg38_multianno.txt
dim: 34355 3191 
[1] "Done !!"


# Check columns

In [6]:
#check column names
colnames(annoDf)
head(annoDf$'CADD_raw')
head(annoDf$'CADD_phred')
print('Done')

[1] "Chr"                                 
   [2] "Start"                               
   [3] "End"                                 
   [4] "Ref"                                 
   [5] "Alt"                                 
   [6] "Func.refGene"                        
   [7] "Gene.refGene"                        
   [8] "GeneDetail.refGene"                  
   [9] "ExonicFunc.refGene"                  
  [10] "AAChange.refGene"                    
  [11] "cytoband"                            
  [12] "SIFT_score"                          
  [13] "SIFT_converted_rankscore"            
  [14] "SIFT_pred"                           
  [15] "Polyphen2_HDIV_score"                
  [16] "Polyphen2_HDIV_rankscore"            
  [17] "Polyphen2_HDIV_pred"                 
  [18] "Polyphen2_HVAR_score"                
  [19] "Polyphen2_HVAR_rankscore"            
  [20] "Polyphen2_HVAR_pred"                 
  [21] "LRT_score"                           
  [22] "LRT_converted_rankscore"             
  [23] "LRT_pred"                            
  [24] "MutationTaster_score"                
  [25] "MutationTaster_converted_rankscore"  
  [26] "MutationTaster_pred"                 
  [27] "MutationAssessor_score"              
  [28] "MutationAssessor_score_rankscore"    
  [29] "MutationAssessor_pred"               
  [30] "FATHMM_score"                        
  [31] "FATHMM_converted_rankscore"          
  [32] "FATHMM_pred"                         
  [33] "PROVEAN_score"                       
  [34] "PROVEAN_converted_rankscore"         
  [35] "PROVEAN_pred"                        
  [36] "VEST3_score"                         
  [37] "VEST3_rankscore"                     
  [38] "MetaSVM_score"                       
  [39] "MetaSVM_rankscore"                   
  [40] "MetaSVM_pred"                        
  [41] "MetaLR_score"                        
  [42] "MetaLR_rankscore"                    
  [43] "MetaLR_pred"                         
  [44] "M-CAP_score"                         
  [45] "M-CAP_rankscore"                     
  [46] "M-CAP_pred"                          
  [47] "REVEL_score"                         
  [48] "REVEL_rankscore"                     
  [49] "MutPred_score"                       
  [50] "MutPred_rankscore"                   
  [51] "CADD_raw"                            
  [52] "CADD_raw_rankscore"                  
  [53] "CADD_phred"                          
  [54] "DANN_score"                          
  [55] "DANN_rankscore"                      
  [56] "fathmm-MKL_coding_score"             
  [57] "fathmm-MKL_coding_rankscore"         
  [58] "fathmm-MKL_coding_pred"              
  [59] "Eigen_coding_or_noncoding"           
  [60] "Eigen-raw"                           
  [61] "Eigen-PC-raw"                        
  [62] "GenoCanyon_score"                    
  [63] "GenoCanyon_score_rankscore"          
  [64] "integrated_fitCons_score"            
  [65] "integrated_fitCons_score_rankscore"  
  [66] "integrated_confidence_value"         
  [67] "GERP++_RS"                           
  [68] "GERP++_RS_rankscore"                 
  [69] "phyloP100way_vertebrate"             
  [70] "phyloP100way_vertebrate_rankscore"   
  [71] "phyloP20way_mammalian"               
  [72] "phyloP20way_mammalian_rankscore"     
  [73] "phastCons100way_vertebrate"          
  [74] "phastCons100way_vertebrate_rankscore"
  [75] "phastCons20way_mammalian"            
  [76] "phastCons20way_mammalian_rankscore"  
  [77] "SiPhy_29way_logOdds"                 
  [78] "SiPhy_29way_logOdds_rankscore"       
  [79] "Interpro_domain"                     
  [80] "GTEx_V6p_gene"                       
  [81] "GTEx_V6p_tissue"                     
  [82] "avsnp150"                            
  [83] "REVEL"                               
  [84] "AF"                                  
  [85] "AF_popmax"                           
  [86] "AF_male"                             
  [87] "AF_female"                           
 

[1] "0.811" "."     "1.376" "1.864" "2.177" "."

[1] "9.530" "."     "12.66" "15.37" "17.36" "."

[1] "Done"


In [16]:
#check values
unique(annoDf$'Func.refGene')
unique(annoDf$'ExonicFunc.refGene')
range(annoDf$'CADD_raw')
'.'%in%annoDf$'CADD_raw'
head(annoDf$'CADD_raw')

[1] "exonic"              "intronic"            "UTR5"               
 [4] "UTR3"                "upstream"            "ncRNA_exonic"       
 [7] "ncRNA_intronic"      "downstream"          "intergenic"         
[10] "splicing"            "upstream;downstream" "exonic;splicing"

[1] "nonsynonymous SNV"      "synonymous SNV"         "stopgain"              
[4] "."                      "nonframeshift deletion" "frameshift deletion"   
[7] "startloss"              "frameshift insertion"

[1] "-0.023" "9.774"

[1] TRUE

[1] "0.811" "."     "1.376" "1.864" "2.177" "."

# Copy LSD genes

In [28]:
#copy the LSD genes
cmd = str_glue('gsutil cp gs://{googleGSId}/notebooks/LSD_genes_080219.txt {dataDir}')
cat('cmd:', cmd, '\n')
shell_do(cmd)

cmd: gsutil cp gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5/notebooks/LSD_genes_080219.txt /home/jupyter-user/notebooks/LSDProject/data/ 
[1] "Executing:  gsutil cp gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5/notebooks/LSD_genes_080219.txt /home/jupyter-user/notebooks/LSDProject/data/"


character(0)

[1] 57  2

Lysosomal Storage Disorder,Gene
<chr>,<chr>
Aspartylglucosaminuria,AGA
Metachromatic leukodystrophy,ARSA
Maroteaux-Lamy disease,ARSB
Farber Lipogranulomatosis,ASAH1
Kufor-Rakeb Syndrome,ATP13A2
Neuronal ceroid lipofuscinosis (CLN3),CLN3


[1] "Lysosomal Storage Disorder" "Gene"

[1] "Done"


# Read LSD gene DF

In [11]:
#read the geneDf
cat('dataDir:', dataDir, '\n')
#fileName=paste0(dataDir,'LSD_genes_080219.txt')
#these are the 51 genes from the paper where the 3 genes "CLN5"  "NEU1"  "SUMF1" where not included since in the 
#paper were not included
fileName=paste0(dataDir,'LSDGenes_51_genes_paperSupplement.txt')
geneDf=fread(fileName,  header=T, sep='\t',stringsAsFactors = F, check.names=F)
geneDf=as.data.frame(geneDf)
dim(geneDf)
head(geneDf)
colnames(geneDf)
cat('length:', length(geneDf$Gene), '\n')
cat('length:', length(unique(geneDf$Gene)), '\n')
cat('Done\n')

dataDir: /home/jupyter-user/notebooks/LSDProject/data/ 


[1] 51  1

,Gene
,<chr>
1,ATP13A2
2,FUCA1
3,PPT1
4,CTSK
5,GBA
6,ST3GAL5


[1] "Gene"

length: 51 
length: 51 
Done


# Find missing genes

In [8]:
#find any missing 
fileName=paste0(dataDir,'/LSD_genes_080219.txt')
geneDf=fread(fileName,  header=T, sep='\t',stringsAsFactors = F, check.names=F)
dim(geneDf)
colnames(geneDf)
#all(geneDf$Gene %in% annoDf$'Gene.refGene')
d=setdiff(geneDf$Gene,annoDf$'Gene.refGene')
d#'GLA''IDS''LAMP2' missing genes from annotation

[1] 57  2

[1] "Lysosomal Storage Disorder" "Gene"

[1] "GLA"   "IDS"   "LAMP2" ""

# Get number of variants per gene and write

In [27]:
#get number of variants per gene


if(1)
{
    #go thru each gene and see how many variants per gene
    geneVec=c()
    allVec=c()
    nonSynVec=c()
    caddVec=c()
    minCaddScore=12.37
    for(geneName in geneDf$Gene)
    {
        if(!(geneName%in% annoDf$'Gene.refGene')){next}
        print(str_glue('Gene:{geneName}'))
        geneVec=append(geneVec, geneName)
        #filter to gene and get all varaints
        allDf=annoDf[annoDf$'Gene.refGene'==geneName,]
        n=nrow(allDf)
        print(str_glue('total numvars:{n}'))
        allVec=append(allVec, n)
        #filter to non synonmous: 
        #column ExonicFunc.refGene: filter out synonymous.
        #column Func.refGene:exonic and splicing only
        #selVec=c('exonic', 'splicing', 'exonic;splicing')
        #nonSynDf=allDf[allDf$'ExonicFunc.refGene'!='synonymous SNV',]
        nonSynDf=allDf[allDf$'ExonicFunc.refGene'=='nonsynonymous SNV',]
        n=nrow(nonSynDf)
        print(str_glue('nonSynDf total numvars:{n}'))
        selVec=c('exonic')
        nonSynDf=nonSynDf[nonSynDf$'Func.refGene'%in%selVec,]
        n=nrow(nonSynDf)
        print(str_glue('nonSyn and exonic total numvars:{n}'))
        nonSynVec=append(nonSynVec, n)
        #filter using CADD from paper , >=12.37
        #first remove all .
        caddDf=allDf
        caddDf$'CADD_phred'=as.numeric(caddDf$'CADD_phred')
        caddDf[is.na(caddDf)]= 0#for the .
        print(head(caddDf$'CADD_phred'))
        cat('range:', range(caddDf$'CADD_phred'), '\n')
        caddDf=caddDf[caddDf$'CADD_phred'>=minCaddScore,]
        n=nrow(caddDf)
        print(str_glue('CADD total numvars:{n}'))
        caddVec=append(caddVec, n)
        #break
    }

    countDf=data.frame('Gene'=geneVec,'numVariants'=allVec,
                       'nonSynExonicNumVars'=nonSynVec,'CADDNumVars'=caddVec,
                       stringsAsFactors = F, check.names=F)       
    print(head(countDf))
    print('Done!!')
}

Gene:ATP13A2
total numvars:283
nonSynDf total numvars:26
nonSyn and exonic total numvars:26


Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


[1]  9.53  0.00 12.66 15.37 17.36  0.00
range: 0 37 
CADD total numvars:20
Gene:FUCA1
total numvars:191
nonSynDf total numvars:12
nonSyn and exonic total numvars:12


Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


[1] 0 0 0 0 0 0
range: 0 28.4 
CADD total numvars:11
Gene:PPT1
total numvars:269
nonSynDf total numvars:5
nonSyn and exonic total numvars:5


...

[1] 0 0 0 0 0 0
range: 0 0 
CADD total numvars:0
Gene:NAGA
total numvars:109
nonSynDf total numvars:10
nonSyn and exonic total numvars:10


Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


[1] 0 0 0 0 0 0
range: 0 34 
CADD total numvars:10
Gene:ARSA
total numvars:78
nonSynDf total numvars:13
nonSyn and exonic total numvars:13


Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


[1] 0 0 0 0 0 0
range: 0 35 
CADD total numvars:11
     Gene numVariants nonSynExonicNumVars CADDNumVars
1 ATP13A2         283                  26          20
2   FUCA1         191                  12          11
3    PPT1         269                   5           4
4    CTSK          75                   5           3
5     GBA          98                  22          20
6 ST3GAL5         488                   7           5
[1] "Done!!"


In [29]:
#write
fileName=str_glue('{dataDir}LSD-genes_num_vars-all-NonSyn-Exonic-CADD.txt')
write.table(countDf, file=fileName, sep='\t', row.names = F, quote = F)

print('Done!!')


[1] "Done!!"


# Copy to bucket number of variants for checking

In [10]:
fileName=str_glue('{dataDir}LSD-genes_num_vars-all-NonSyn-Exonic-CADD.txt')
fileName=str_glue('{resDir}{annoPrefix}.annovar.hg38_multianno.txt')
cmd=str_glue('gsutil -u {BILLING_PROJECT_ID} cp {fileName} {WORKSPACE_BUCKET}')
cat('cmd:', cmd, '\n')
shell_do(cmd)
print('Done')


cmd: gsutil -u larobak11 cp /home/jupyter-user/notebooks/LSDProject/results/LSDGenes-WGS_Dec3120/LSD-GENES.annovar.hg38_multianno.txt gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5 
[1] "Executing:  gsutil -u larobak11 cp /home/jupyter-user/notebooks/LSDProject/results/LSDGenes-WGS_Dec3120/LSD-GENES.annovar.hg38_multianno.txt gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5"


character(0)

[1] "Done"


In [43]:
#copy
fileName='../LSD-All-genes/stats/LSD-genes_num_vars-all-NonSyn-CADD.txt'
cat('outFile:', fileName,'\n')
cmd=str_glue('gsutil -u {BILLING_PROJECT_ID} cp -r ./{fileName} {WORKSPACE_BUCKET}')
shell_do(cmd)
print('finished copying')

outFile: ../LSD-All-genes/stats/LSD-genes_num_vars-all-NonSyn-CADD.txt 
[1] "Executing:  gsutil -u fccredits-lithium-fuchsia-8036 cp -r ./../LSD-All-genes/stats/LSD-genes_num_vars-all-NonSyn-CADD.txt gs://fc-a81f9f10-5bf2-4ac0-bc8f-86c6f7231ec9"


character(0)

[1] "finished copying"


# Make positon file with exon / nonsyn

In [55]:
#filter the files to non-synonomys/exonic and CADD filter only and get SNP pos for plink
annoPrefix='LSD-GENES'
annoFile=paste0(resDir,annoPrefix,'.annovar.hg38_multianno.txt')
cat('annoFile:',annoFile, '\n')
#filter to LSD gene names
cat('num LSD genes:', length(geneDf$Gene),' ', length(geneDf$Gene), '\n')
filtDf=annoDf[annoDf$'Gene.refGene'%in%geneDf$Gene,]
cat('nrows:', nrow(filtDf), '\n')
n=length(unique(filtDf$'Gene.refGene'))
cat('n:', n,'\n')
d1=setdiff(geneDf$Gene, unique(filtDf$'Gene.refGene'))
cat('d1:', d1, '\n')


#filter to non synonmous:
#selVec=c('exonic', 'splicing', 'exonic;splicing')
selVec=c('exonic')
#filtDf=filtDf[filtDf$'ExonicFunc.refGene'!='synonymous SNV',]
filtDf=filtDf[filtDf$'ExonicFunc.refGene'=='nonsynonymous SNV',]
filtDf=filtDf[filtDf$'Func.refGene'%in%selVec,]
n=nrow(filtDf)
print(str_glue('nonSyn total numvars:{n}'))
#write the list of variants which pass this threshold
fileName=str_glue('{dataDir}LSD-Genes_SNP-pos_nonSynExonicFilt_allColumns.txt')
print(fileName)
write.table(filtDf, file=fileName, sep='\t', row.names = F,col.names=F, quote = F)
#shell_do(f'cut -f 1,2,3,7  {workingDir}/CODING_regions.txt > {workingDir}/CODING.txt')
filtDf=filtDf[, c(1,2,3,7)]
#check head
print(head(filtDf))
cat('num Genes:', length(unique(filtDf$Gene.refGene)), '\n')
d1=setdiff(geneDf$Gene, unique(filtDf$Gene.refGene))
cat('d1:', d1, '\n')
#write
fileName=str_glue('{dataDir}LSD-Genes_SNP-pos_nonSynExonicFilt.txt')
print(fileName)
write.table(filtDf, file=fileName, sep='\t', row.names = F,col.names=F, quote = F)

print('Done!!')


annoFile: /home/jupyter-user/notebooks/LSDProject/results/LSDGenes-WGS_Dec3120/LSD-GENES.annovar.hg38_multianno.txt 
num LSD genes: 51   51 
nrows: 18188 
n: 48 
d1: GLA LAMP2 IDS 
nonSyn total numvars:649
/home/jupyter-user/notebooks/LSDProject/data/LSD-Genes_SNP-pos_nonSynExonicFilt_allColumns.txt
   Chr    Start      End Gene.refGene
1:   1 16985990 16985990      ATP13A2
2:   1 16986091 16986091      ATP13A2
3:   1 16986097 16986097      ATP13A2
4:   1 16986101 16986101      ATP13A2
5:   1 16986246 16986246      ATP13A2
6:   1 16986292 16986292      ATP13A2
num Genes: 47 
d1: DNAJC5 GLA LAMP2 IDS 
/home/jupyter-user/notebooks/LSDProject/data/LSD-Genes_SNP-pos_nonSynExonicFilt.txt
[1] "Done!!"


In [17]:
#copy to bucket
fileName=str_glue('{dataDir}LSD-Genes_SNP-pos_nonSynExonicFilt_allColumns.txt')
cat('outFile:', fileName,'\n')
cmd=str_glue('gsutil -u {BILLING_PROJECT_ID} cp -r {fileName} {WORKSPACE_BUCKET}')
cat('cmd:', cmd, '\n')
shell_do(cmd)
print('finished copying')

outFile: /home/jupyter-user/notebooks/LSDProject/data/LSD-Genes_SNP-pos_nonSynExonicFilt_allColumns.txt 
cmd: gsutil -u larobak11 cp -r /home/jupyter-user/notebooks/LSDProject/data/LSD-Genes_SNP-pos_nonSynExonicFilt_allColumns.txt gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5 
[1] "Executing:  gsutil -u larobak11 cp -r /home/jupyter-user/notebooks/LSDProject/data/LSD-Genes_SNP-pos_nonSynExonicFilt_allColumns.txt gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5"


character(0)

[1] "finished copying"


# Prep files for plink and generate/extract PLINK binary with only non syn variants

In [56]:
#Prep the files and generate PLINK binary with only non syn variants
bFile=str_glue('{dataDir}LSDGenes-WGS-AMPPD')
outPrefix=str_glue('{dataDir}/LSDGenes-WGS-AMPPD_nonSynExonicFilt')
phenoFile=str_glue('{dataDir}plink_pheno_withPCs.tab')
varPosFile=str_glue('{dataDir}/LSD-Genes_SNP-pos_nonSynExonicFilt.txt')
#shell_do(f'~/bin/plink --bfile {bFile} --pheno {phenoFile} --pheno-name PHENO --extract range {workingDir}/CODING.txt --make-bed --out {outPrefix}')
cmd=str_glue('{plinkCmd} --bfile {bFile} --pheno {phenoFile} --pheno-name PHENO --extract range {varPosFile} --make-bed --out {outPrefix}')
cat('cmd:',cmd,'\n')
shell_do(cmd)
print('Done !!')

cmd: /home/jupyter-user/notebooks/LSDProject/tools/plink --bfile /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs.tab --pheno-name PHENO --extract range /home/jupyter-user/notebooks/LSDProject/data//LSD-Genes_SNP-pos_nonSynExonicFilt.txt --make-bed --out /home/jupyter-user/notebooks/LSDProject/data//LSDGenes-WGS-AMPPD_nonSynExonicFilt 
[1] "Executing:  /home/jupyter-user/notebooks/LSDProject/tools/plink --bfile /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs.tab --pheno-name PHENO --extract range /home/jupyter-user/notebooks/LSDProject/data//LSD-Genes_SNP-pos_nonSynExonicFilt.txt --make-bed --out /home/jupyter-user/notebooks/LSDProject/data//LSDGenes-WGS-AMPPD_nonSynExonicFilt"


[1] "PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
 [2] "(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
 [3] "Logging to /home/jupyter-user/notebooks/LSDProject/data//LSDGenes-WGS-AMPPD_nonSynExonicFilt.log."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
 [4] "Options in effect:"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
 [5] "  --bfile /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD"                                                                                                                                                                                                                                                 

[1] "Done !!"


# Create the vcf for the filtered file nonsyn

In [57]:
#create the vcf
prefix='LSDGenes-WGS-AMPPD_nonSynExonicFilt'
bFile=str_glue('{dataDir}{prefix}')
outPrefix=str_glue('{prefix}-VCF')
outPrefix=str_glue('{dataDir}{outPrefix}')

cmd=str_glue('{plinkCmd} --bfile {bFile} --pheno {phenoFile} --pheno-name PHENO --recode vcf-fid --out {outPrefix}')
cat('cmd:', cmd, '\n')
shell_do(cmd)
print('Done !!')

cmd: /home/jupyter-user/notebooks/LSDProject/tools/plink --bfile /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_nonSynExonicFilt --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs.tab --pheno-name PHENO --recode vcf-fid --out /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_nonSynExonicFilt-VCF 
[1] "Executing:  /home/jupyter-user/notebooks/LSDProject/tools/plink --bfile /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_nonSynExonicFilt --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs.tab --pheno-name PHENO --recode vcf-fid --out /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_nonSynExonicFilt-VCF"


[1] "PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
 [2] "(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
 [3] "Logging to /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_nonSynExonicFilt-VCF.log."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
 [4] "Options in effect:"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
 [5] "  --bfile /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_nonSynExonicFilt"                                                                                                                                                                                                                                

[1] "Done !!"


# bgzip and tabix

In [58]:
#outprefix is set up above
cmd=str_glue('bgzip {outPrefix}.vcf')
shell_do(cmd)
cmd=str_glue('tabix -f -p vcf {outPrefix}.vcf.gz')
shell_do(cmd)
cat('Done\n')

[1] "Executing:  bgzip /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_nonSynExonicFilt-VCF.vcf"


character(0)

[1] "Executing:  tabix -f -p vcf /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_nonSynExonicFilt-VCF.vcf.gz"


character(0)

Done


# Prep files for plink and generate/extract PLINK binary with CADD filter

In [59]:
#filter the files to CADD and get SNP pos for plink
annoPrefix='LSD-GENES'
annoFile=paste0(dataDir,'/',annoPrefix,'.annovar.hg38_multianno.txt')
cat('annoFile:',annoFile, '\n')
#filter to LSD gene names
filtDf=annoDf[annoDf$'Gene.refGene'%in%geneDf$Gene,]
cat('nrows:', nrow(filtDf), '\n')
n=length(unique(filtDf$'Gene.refGene'))
cat('n:', n,'\n')


#filter
minCaddScore=12.37
#replace all the . to zero
print(head(filtDf$'CADD_phred'))
filtDf$'CADD_phred'=as.numeric(filtDf$'CADD_phred')
filtDf[is.na(filtDf)]= 0#for the .
print(head(filtDf$'CADD_phred'))
cat('range:', range(filtDf$'CADD_phred'), '\n')
filtDf=filtDf[filtDf$'CADD_phred'>=minCaddScore,]
cat('range:', range(filtDf$'CADD_phred'), '\n')
n=nrow(filtDf)
print(str_glue('CADD total numvars:{n}'))
#write the list of variants which pass this threshold
fileName=str_glue('{dataDir}LSD-Genes_SNP-pos_CADDFilt_allColumns.txt')
print(fileName)
write.table(filtDf, file=fileName, sep='\t', row.names = F,col.names=F, quote = F)

#select columns
filtDf=filtDf[, c(1,2,3,7)]
#check head
print(head(filtDf))
cat('filtDf nrow:', nrow(filtDf), ' numGenes:',length(unique(filtDf$Gene.refGene)), '\n')
d1=setdiff(geneDf$Gene, unique(filtDf$Gene.refGene))
cat('d1:', d1, '\n')
#write
fileName=str_glue('{dataDir}/LSD-Genes_SNP-pos_CADDFilt.txt')
#print(fileName)
write.table(filtDf, file=fileName, sep='\t', row.names = F,col.names=F, quote = F)

print('Done!!')



annoFile: /home/jupyter-user/notebooks/LSDProject/data//LSD-GENES.annovar.hg38_multianno.txt 
nrows: 18188 
n: 48 
[1] "9.530" "."     "12.66" "15.37" "17.36" "."    


Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


[1]  9.53  0.00 12.66 15.37 17.36  0.00
range: 0 43 
range: 12.4 43 
CADD total numvars:474
/home/jupyter-user/notebooks/LSDProject/data/LSD-Genes_SNP-pos_CADDFilt_allColumns.txt
   Chr    Start      End Gene.refGene
1:   1 16986091 16986091      ATP13A2
2:   1 16986097 16986097      ATP13A2
3:   1 16986101 16986101      ATP13A2
4:   1 16986274 16986274      ATP13A2
5:   1 16986292 16986292      ATP13A2
6:   1 16986334 16986334      ATP13A2
filtDf nrow: 474  numGenes: 47 
d1: DNAJC5 GLA LAMP2 IDS 
[1] "Done!!"


In [18]:
#copy to bucket
fileName=str_glue('{dataDir}LSD-Genes_SNP-pos_CADDFilt_allColumns.txt')
cat('outFile:', fileName,'\n')
cmd=str_glue('gsutil -u {BILLING_PROJECT_ID} cp -r {fileName} {WORKSPACE_BUCKET}')
cat('cmd:', cmd, '\n')
shell_do(cmd)
print('finished copying')

outFile: /home/jupyter-user/notebooks/LSDProject/data/LSD-Genes_SNP-pos_CADDFilt_allColumns.txt 
cmd: gsutil -u larobak11 cp -r /home/jupyter-user/notebooks/LSDProject/data/LSD-Genes_SNP-pos_CADDFilt_allColumns.txt gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5 
[1] "Executing:  gsutil -u larobak11 cp -r /home/jupyter-user/notebooks/LSDProject/data/LSD-Genes_SNP-pos_CADDFilt_allColumns.txt gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5"


character(0)

[1] "finished copying"


# Extract for the filtered file CADD

In [60]:
#Prep the files and generate PLINK binary with CADD filtered filter
bFile=str_glue('{dataDir}LSDGenes-WGS-AMPPD')
outPrefix=str_glue('{dataDir}LSDGenes-WGS-AMPPD_CADDFilt')
phenoFile=str_glue('{dataDir}plink_pheno_withPCs.tab')
varPosFile=str_glue('{dataDir}LSD-Genes_SNP-pos_CADDFilt.txt')
cmd=str_glue('{plinkCmd} --bfile {bFile} --pheno {phenoFile} --pheno-name PHENO --extract range {varPosFile} --make-bed --out {outPrefix}')
cat('cmd:', cmd, '\n')
shell_do(cmd)
print('Done !!')

cmd: /home/jupyter-user/notebooks/LSDProject/tools/plink --bfile /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs.tab --pheno-name PHENO --extract range /home/jupyter-user/notebooks/LSDProject/data/LSD-Genes_SNP-pos_CADDFilt.txt --make-bed --out /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_CADDFilt 
[1] "Executing:  /home/jupyter-user/notebooks/LSDProject/tools/plink --bfile /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs.tab --pheno-name PHENO --extract range /home/jupyter-user/notebooks/LSDProject/data/LSD-Genes_SNP-pos_CADDFilt.txt --make-bed --out /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_CADDFilt"


[1] "PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
 [2] "(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
 [3] "Logging to /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_CADDFilt.log."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
 [4] "Options in effect:"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
 [5] "  --bfile /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD"                                                                                                                                                                                                                                                 

[1] "Done !!"


# Create the vcf

In [61]:
#create the vcf
prefix='LSDGenes-WGS-AMPPD_CADDFilt'
bFile=str_glue('{dataDir}{prefix}')
outPrefix=str_glue('{prefix}-VCF')
outPrefix=str_glue('{dataDir}{outPrefix}')

cmd=str_glue('{plinkCmd} --bfile {bFile} --pheno {phenoFile} --pheno-name PHENO --recode vcf-fid --out {outPrefix}')
shell_do(cmd)
print('Done !!')


[1] "Executing:  /home/jupyter-user/notebooks/LSDProject/tools/plink --bfile /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_CADDFilt --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs.tab --pheno-name PHENO --recode vcf-fid --out /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_CADDFilt-VCF"


[1] "PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
 [2] "(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
 [3] "Logging to /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_CADDFilt-VCF.log."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
 [4] "Options in effect:"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
 [5] "  --bfile /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_CADDFilt"                                                                                                                                                                                                                                        

[1] "Done !!"


# bgzip and tabix

In [62]:
#outprefix is set up above
cmd=str_glue('bgzip {outPrefix}.vcf')
shell_do(cmd)
cmd=str_glue('tabix -f -p vcf {outPrefix}.vcf.gz')
shell_do(cmd)
cat('Done\n')

[1] "Executing:  bgzip /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_CADDFilt-VCF.vcf"


character(0)

[1] "Executing:  tabix -f -p vcf /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_CADDFilt-VCF.vcf.gz"


character(0)

Done


# Burden test

In [11]:
#burden test 
#set up Dirs
#workDir='/home/jupyter-user/notebooks/LaurieRobak1/LSD-All-genes'
#dataDir='/home/jupyter-user/notebooks/LaurieRobak1/data'
#rvtestCmd='/home/jupyter-user/notebooks/LaurieRobak1/tools/executable/rvtest'
#phenoFile='/home/jupyter-user/notebooks/LaurieRobak1/data/plink_test_pheno_withPCs_rvtestFormat.tab'
dataDir='/home/jupyter-user/notebooks/LSDProject/data/'
rvtestCmd='/home/jupyter-user/notebooks/LSDProject/tools/rvtests/executable/rvtest'
#phenoFile='/home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat.tab'
phenoFile='/home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat_allRaces.tab'
#phenoFile='/home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat_allRaces_PPMI.tab'
#COV_NAMES="sex,AgeBaseLine,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10"
COV_NAMES="sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20"
#select maf cut
mafCut=0.03
#mafCut=0.01
if(mafCut==0.03)
{
  mafCutStr='maf003'  
}else if(mafCut==0.01)
{
  mafCutStr='maf001'
}
cat('mafcutStr:', mafCutStr, '\n')
    
#result dir
#resDir='/home/jupyter-user/notebooks/LSDProject/results/res_Jan0620_burden/'
#resDir='/home/jupyter-user/notebooks/LSDProject/results/tmp/'
#resDir='/home/jupyter-user/notebooks/LSDProject/results/res_Feb1021_47LSDGenes/'
resDir='/home/jupyter-user/notebooks/LSDProject/results/res_Feb1021_47LSDGenes_usingSetFile/'


#select the type of input based on filter
#filterType='nonSynFilt'
#filterType='nonSynExonicFilt'
filterType='CADDFilt'
cat('filterType:', filterType, '\n')
inputPrefix=str_glue('LSDGenes-WGS-AMPPD_{filterType}-VCF')
vcfFile=str_glue('{dataDir}{inputPrefix}.vcf.gz')
cat('vcfFile:', vcfFile, '\n')

#studyName='PPMI'

#select set file
#gbaFlag='withGBA'
gbaFlag='withoutGBA'
if(gbaFlag=='withGBA')
{
    #setFile=glue_str('{workDir}/LSD-53-genes-withoutGBA_rvtest_SetFile.txt')
    #setName='LSDGenes53'
    setFile=str_glue('{dataDir}LSD-54-genes_rvtest_SetFile.txt')
    setName='LSDGenes54'

    nameStr=str_glue('LSDGenes-{gbaFlag}_WGS_AMPPD_BURDEN_{filterType}_Variants')
    outPrefix=str_glue('{resDir}{nameStr}.{mafCutStr}')
    #outPrefix=str_glue('{resDir}{nameStr}_{studyName}.{mafCutStr}')
}else if(gbaFlag=='withoutGBA')
{
    setFile=str_glue('{dataDir}LSD-53-genes-withoutGBA_rvtest_SetFile.txt')
    setName='LSDGenes53'
    nameStr=str_glue('LSDGenes-{gbaFlag}_WGS_AMPPD_BURDEN_{filterType}_Variants')
    outPrefix=str_glue('{resDir}{nameStr}.{mafCutStr}')
    #outPrefix=str_glue('{resDir}{nameStr}_{studyName}.{mafCutStr}')
}

cmd=str_glue('{rvtestCmd} --noweb --hide-covar --inVcf {vcfFile} ',
'--pheno {phenoFile} ' ,
'--pheno-name PHENO ' ,
'--covar {phenoFile} ',
'--covar-name {COV_NAMES} ',
'--kernel skato ',
'--setFile {setFile} ',
'--set {setName} ',
'--freqUpper {mafCut} ',
'--out {outPrefix}'
)
print(cmd)
shell_do(cmd)
print('Done!!')   


mafcutStr: maf003 
filterType: CADDFilt 
vcfFile: /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_CADDFilt-VCF.vcf.gz 
/home/jupyter-user/notebooks/LSDProject/tools/rvtests/executable/rvtest --noweb --hide-covar --inVcf /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_CADDFilt-VCF.vcf.gz --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat_allRaces.tab --pheno-name PHENO --covar /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat_allRaces.tab --covar-name sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20 --kernel skato --setFile /home/jupyter-user/notebooks/LSDProject/data/LSD-53-genes-withoutGBA_rvtest_SetFile.txt --set LSDGenes53 --freqUpper 0.03 --out /home/jupyter-user/notebooks/LSDProject/results/res_Feb1021_47LSDGenes_usingSetFile/LSDGenes-withoutGBA_WGS_AMPPD_BURDEN_CADDFilt_Variants.maf003
[1] "Executing:  /home/jupyter-user/notebooks/LSDProject

[1] "Thank you for using rvtests (version: 20190205, git: c86e589efef15382603300dc7f4c3394c82d69b8)"
[2] "  For documentations, refer to http://zhanxw.github.io/rvtests/"                               
[3] "  For questions and comments, plase send to Xiaowei Zhan <zhanxw@umich.edu>"                   
[4] "  For bugs and feature requests, please submit at: https://github.com/zhanxw/rvtests/issues"   
[5] ""                                                                                              
[6] "RVTESTS finished successfully"

[1] "Done!!"


# Burden test but using refFlat file for all genes individually 

In [19]:
#burden test 
#set up Dirs
#workDir='/home/jupyter-user/notebooks/LaurieRobak1/LSD-All-genes'
#dataDir='/home/jupyter-user/notebooks/LaurieRobak1/data'
#rvtestCmd='/home/jupyter-user/notebooks/LaurieRobak1/tools/executable/rvtest'
#phenoFile='/home/jupyter-user/notebooks/LaurieRobak1/data/plink_test_pheno_withPCs_rvtestFormat.tab'
dataDir='/home/jupyter-user/notebooks/LSDProject/data/'
rvtestCmd='/home/jupyter-user/notebooks/LSDProject/tools/rvtests/executable/rvtest'
#phenoFile='/home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat.tab'
phenoFile='/home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat_allRaces.tab'
#phenoFile='/home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat_allRaces_PPMI.tab'
#COV_NAMES="sex,AgeBaseLine,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10"
COV_NAMES="sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20"
#select maf cut
mafCut=0.03
#mafCut=0.01
if(mafCut==0.03)
{
  mafCutStr='maf003'  
}else if(mafCut==0.01)
{
  mafCutStr='maf001'
}
cat('mafcutStr:', mafCutStr, '\n')
    
#result dir
#resDir='/home/jupyter-user/notebooks/LSDProject/results/res_Jan0620_burden/'
#resDir='/home/jupyter-user/notebooks/LSDProject/results/tmp/'
#resDir='/home/jupyter-user/notebooks/LSDProject/results/res_Feb1021_47LSDGenes/'
resDir='/home/jupyter-user/notebooks/LSDProject/results/res_Feb1021_47LSDGenes_usingRefFlatFile/'


#select the type of input based on filter
#filterType='nonSynFilt'
#filterType='nonSynExonicFilt'
filterType='CADDFilt'
inputPrefix=str_glue('LSDGenes-WGS-AMPPD_{filterType}-VCF')
vcfFile=str_glue('{dataDir}{inputPrefix}.vcf.gz')
cat('vcfFile:', vcfFile, '\n')

#studyName='PPMI'

#select set file
gbaFlag='withGBA'
#gbaFlag='withoutGBA'
if(gbaFlag=='withGBA')
{
    #setFile=glue_str('{workDir}/LSD-53-genes-withoutGBA_rvtest_SetFile.txt')
    #setName='LSDGenes53'
    #setFile=str_glue('{dataDir}LSD-54-genes_rvtest_SetFile.txt')
    #setName='LSDGenes54'
    nameStr=str_glue('LSDGenes-{gbaFlag}_WGS_AMPPD_BURDEN_{filterType}_Variants')
    outPrefix=str_glue('{resDir}{nameStr}.{mafCutStr}')
    #outPrefix=str_glue('{resDir}{nameStr}_{studyName}.{mafCutStr}')
}else if(gbaFlag=='withoutGBA')
{
    #setFile=str_glue('{dataDir}LSD-53-genes-withoutGBA_rvtest_SetFile.txt')
    #setName='LSDGenes53'
    nameStr=str_glue('LSDGenes-{gbaFlag}_WGS_AMPPD_BURDEN_{filterType}_Variants')
    outPrefix=str_glue('{resDir}{nameStr}.{mafCutStr}')
    #outPrefix=str_glue('{resDir}{nameStr}_{studyName}.{mafCutStr}')
}

    
refFlatFile='/home/jupyter-user/notebooks/LSDProject/data/refFlat_hg38.txt'
    
cmd=str_glue('{rvtestCmd} --noweb --hide-covar --inVcf {vcfFile} ',
'--pheno {phenoFile} ' ,
'--pheno-name PHENO ' ,
'--covar {phenoFile} ',
'--covar-name {COV_NAMES} ',
'--kernel skato ',
'--geneFile {refFlatFile} ',
'--freqUpper {mafCut} ',
'--out {outPrefix}'
)
print(cmd)
shell_do(cmd)
print('Done!!')   


mafcutStr: maf003 
vcfFile: /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_CADDFilt-VCF.vcf.gz 
/home/jupyter-user/notebooks/LSDProject/tools/rvtests/executable/rvtest --noweb --hide-covar --inVcf /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_CADDFilt-VCF.vcf.gz --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat_allRaces.tab --pheno-name PHENO --covar /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat_allRaces.tab --covar-name sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20 --kernel skato --geneFile /home/jupyter-user/notebooks/LSDProject/data/refFlat_hg38.txt --freqUpper 0.03 --out /home/jupyter-user/notebooks/LSDProject/results/res_Feb1021_47LSDGenes_usingRefFlatFile/LSDGenes-withGBA_WGS_AMPPD_BURDEN_CADDFilt_Variants.maf003
[1] "Executing:  /home/jupyter-user/notebooks/LSDProject/tools/rvtests/executable/rvtest --noweb --hide-covar --inVcf /

[1] "Thank you for using rvtests (version: 20190205, git: c86e589efef15382603300dc7f4c3394c82d69b8)"
[2] "  For documentations, refer to http://zhanxw.github.io/rvtests/"                               
[3] "  For questions and comments, plase send to Xiaowei Zhan <zhanxw@umich.edu>"                   
[4] "  For bugs and feature requests, please submit at: https://github.com/zhanxw/rvtests/issues"   
[5] ""                                                                                              
[6] "RVTESTS finished successfully"

[1] "Done!!"


# Burden using varaint set file per gene from paper supplement file

In [8]:
#burden test 
#set up Dirs
#workDir='/home/jupyter-user/notebooks/LaurieRobak1/LSD-All-genes'
#dataDir='/home/jupyter-user/notebooks/LaurieRobak1/data'
#rvtestCmd='/home/jupyter-user/notebooks/LaurieRobak1/tools/executable/rvtest'
#phenoFile='/home/jupyter-user/notebooks/LaurieRobak1/data/plink_test_pheno_withPCs_rvtestFormat.tab'
dataDir='/home/jupyter-user/notebooks/LSDProject/data/'
rvtestCmd='/home/jupyter-user/notebooks/LSDProject/tools/rvtests/executable/rvtest'
#phenoFile='/home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat.tab'
phenoFile='/home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat_allRaces.tab'
#phenoFile='/home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat_allRaces_PPMI.tab'
#COV_NAMES="sex,AgeBaseLine,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10"
COV_NAMES="sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20"
#select maf cut
mafCut=0.03
#mafCut=0.01
if(mafCut==0.03)
{
  mafCutStr='maf003'  
}else if(mafCut==0.01)
{
  mafCutStr='maf001'
}
cat('mafcutStr:', mafCutStr, '\n')
    
#result dir
#resDir='/home/jupyter-user/notebooks/LSDProject/results/res_Jan0620_burden/'
#resDir='/home/jupyter-user/notebooks/LSDProject/results/tmp/'
#resDir='/home/jupyter-user/notebooks/LSDProject/results/res_Feb1021_47LSDGenes/'
resDir='/home/jupyter-user/notebooks/LSDProject/results/res_Feb1021_using_PaperSupplFile_SetFile/'


#select the type of input based on filter
#filterType='nonSynFilt'
#filterType='nonSynExonicFilt'
filterType='CADDFilt'
cat('filterType:', filterType, '\n')
inputPrefix=str_glue('LSDGenes-WGS-AMPPD_{filterType}-VCF')
vcfFile=str_glue('{dataDir}{inputPrefix}.vcf.gz')
cat('vcfFile:', vcfFile, '\n')


setFile=str_glue('{dataDir}paper_suppl_var_setFile.txt')
setDf=read.table(setFile,  header=F, sep='\t',stringsAsFactors = F, check.names=F)
cat('nrow setDf:', nrow(setDf), '\n')
colnames(setDf)=c('gene', 'set')
for(setName in setDf$gene)
{
    cat('\nsetName:', setName, '\n')
    nameStr=str_glue('LSDGenes_WGS_AMPPD_BURDEN_gene-{setName}_{filterType}_Variants')
    outPrefix=str_glue('{resDir}{nameStr}.{mafCutStr}')    

    cmd=str_glue('{rvtestCmd} --noweb --hide-covar --inVcf {vcfFile} ',
    '--pheno {phenoFile} ' ,
    '--pheno-name PHENO ' ,
    '--covar {phenoFile} ',
    '--covar-name {COV_NAMES} ',
    '--kernel skato ',
    '--setFile {setFile} ',
    '--set {setName} ',
    '--freqUpper {mafCut} ',
    '--out {outPrefix}'
    )
    print(cmd)
    shell_do(cmd)
}

print('Done!!')   


mafcutStr: maf003 
filterType: CADDFilt 
vcfFile: /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_CADDFilt-VCF.vcf.gz 
nrow setDf: 51 
setName: ATP13A2 
/home/jupyter-user/notebooks/LSDProject/tools/rvtests/executable/rvtest --noweb --hide-covar --inVcf /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_CADDFilt-VCF.vcf.gz --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat_allRaces.tab --pheno-name PHENO --covar /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat_allRaces.tab --covar-name sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20 --kernel skato --setFile /home/jupyter-user/notebooks/LSDProject/data/paper_suppl_var_setFile.txt --set ATP13A2 --freqUpper 0.03 --out /home/jupyter-user/notebooks/LSDProject/results/res_Feb1021_using_PaperSupplFile_SetFile/LSDGenes_WGS_AMPPD_BURDEN_gene-ATP13A2_CADDFilt_Variants.maf003
[1] "Executing:  /home/jupyter-us

# Read set file results from section above

In [29]:
resDir='/home/jupyter-user/notebooks/LSDProject/results/res_Feb1021_using_PaperSupplFile_SetFile/'
fileList=list.files(resDir)
length(fileList)
fileList=grep('assoc', fileList, value = T)
length(fileList)

geneVec=c()
numVarVec=c()
pvalVec=c()
for(f in fileList)
{
    #f='LSDGenes_WGS_AMPPD_BURDEN_gene-GBA_CADDFilt_Variants.maf003.SkatO.assoc'
    s = unlist(strsplit(f, '_'))
    geneName=s[5]
    s = unlist(strsplit(geneName, '-'))
    geneName=s[2]
    
    fileName=str_glue('{resDir}{f}')
    df=read.table(fileName,  header=T, sep='\t',stringsAsFactors = F, check.names=F)
    if(nrow(df)==0){next}
    #cat('colnames:', colnames(df), '\n')
    geneVec=c(geneVec, geneName)
    numVarVec=c(numVarVec,df$NumVar)
    pvalVec=c(pvalVec, df$Pvalue)
    #cat('gene:', geneName, df$NumVar, ' ',  df$Pvalue,'\n')
    #break
}
cat(length(geneVec), ' ', length(numVarVec), ' ', length(pvalVec), '\n' )
outDf=data.frame('Gene'=geneVec,'numVar'=numVarVec, 'pval'=pvalVec, stringsAsFactors=F)
#print(outDf)

fileName=str_glue('{dataDir}setfile_per_gene_from_paper_supplement_GeneNumVarsPval.txt')
cat('file:', fileName, '\n')
write.table(outDf, file=fileName, sep='\t', row.names = F, quote = F)



[1] 102

[1] 51

42   42   42 
file: /home/jupyter-user/notebooks/LSDProject/data/setfile_per_gene_from_paper_supplement_GeneNumVarsPval.txt 


# Combine files

In [73]:
#this file has the results of running burden test using the set list of varaints as taken from the supplement paper
fileName=str_glue('{dataDir}setfile_per_gene_from_paper_supplement_GeneNumVarsPval.txt')
paperSetDf=read.table(fileName,  header=T, sep='\t',stringsAsFactors = F, check.names=F)
cat('nrow paperSetDf:', nrow(paperSetDf), '\n')
print(colnames(paperSetDf))

#this file has the burden test using the gene names from the supplement table and using the refFlat38 gene file
fileName='/home/jupyter-user/notebooks/LSDProject/results/res_Feb1021_47LSDGenes_usingRefFlatFile/LSDGenes-withGBA_WGS_AMPPD_BURDEN_CADDFilt_Variants.maf003.SkatO.assoc'
amppdLSDGenesDf=read.table(fileName,  header=T, sep='\t',stringsAsFactors = F, check.names=F)
cat('norw amppdLSDGenesDf:', nrow(amppdLSDGenesDf), '\n')
print(colnames(amppdLSDGenesDf))

#this file is the actual supplement file from the paper which has the varaints per gene
fileName=str_glue('{dataDir}Robak_supp_table_2_withHG38Pos_rsId.txt')
suppDf=read.table(fileName,  header=T, sep='\t',stringsAsFactors = F, check.names=F)
cat('norw suppDf:', nrow(suppDf), '\n')
#get num vars in it
geneVec=c()
numVec=c()
for(geneName in unique(suppDf$Gene) )
{
   filtDf=suppDf[suppDf$Gene==geneName,]
   numVars=nrow(filtDf)
   geneVec=c(geneVec, geneName)
   numVec=c(numVec,numVars)
}
suppNumDf=data.frame('Gene'=geneVec, 'numVars'=numVec, stringsAsFactors=F)
#print(head(suppNumDf), '\n')
#head(suppNumDf)
#dim(suppNumDf)
print(colnames(suppNumDf))

cmnGenes=intersect(intersect(suppNumDf$Gene,amppdLSDGenesDf$Gene),paperSetDf$Gene)
cat('length:', length(cmnGenes), '\n')
#cat('cmnGenes:', cmnGenes, '\n')

#filter and put all in same order
paperSetDf=paperSetDf[paperSetDf$Gene%in%cmnGenes,]
amppdLSDGenesDf=amppdLSDGenesDf[amppdLSDGenesDf$Gene%in%cmnGenes,]
suppNumDf=suppNumDf[suppNumDf$Gene%in%cmnGenes,]

amppdLSDGenesDf=amppdLSDGenesDf[match(paperSetDf$Gene,amppdLSDGenesDf$Gene),]
suppNumDf=suppNumDf[match(amppdLSDGenesDf$Gene,suppNumDf$Gene),]

check_1=all(amppdLSDGenesDf$Gene == paperSetDf$Gene)
check_2=all(suppNumDf$Gene == amppdLSDGenesDf$Gene)
check_3=all(suppNumDf$Gene == amppdLSDGenesDf$Gene)

cat(check_1, ' ', check_2, ' ', check_3, '\n')


suppNumVarsVec=c()
geneVec=c()
paperSetNumVarsVec=c()
paperSetPvalVec=c()
amppdNumVars=c()
amppdPval=c()
for(rowNum in c(1:nrow(suppNumDf))  ) 
{
    geneName=suppNumDf[rowNum,]$Gene
    suppNumVars=suppNumDf[rowNum,]$numVars
    suppNumVarsVec=c(suppNumVarsVec, suppNumVars)
    geneVec=c(geneVec, geneName)
    #
    paperSetNumVarsVec=c(paperSetNumVarsVec,paperSetDf[paperSetDf$Gene==geneName,]$numVar)
    paperSetPvalVec=c(paperSetPvalVec,paperSetDf[paperSetDf$Gene==geneName,]$pval)
    #
    amppdNumVars=c(amppdNumVars, amppdLSDGenesDf[amppdLSDGenesDf$Gene==geneName,]$NumVar)
    amppdPval=c(amppdPval, amppdLSDGenesDf[amppdLSDGenesDf$Gene==geneName,]$Pvalue)
    #break
}

    
outDf=data.frame('Gene'=geneVec,'paper_NumVars'=suppNumVarsVec, 
                 'paper_NumVars_FoundinAMPPD'=paperSetNumVarsVec,
                 'AMPPD_Gene_burden_Pvalue_using_paper_variants'=paperSetPvalVec,
                 'AMPPD_numVars'=amppdNumVars,
                 'AMPPD_burden_pvalue'=amppdPval,
                 stringsAsFactors=F)
print(outDf)

fileName=str_glue('{dataDir}cmp_variants_across_cohorts.txt')
write.table(outDf, file=fileName, sep='\t', row.names = F, quote = F)

nrow paperSetDf: 42 
[1] "Gene"   "numVar" "pval"  
norw amppdLSDGenesDf: 48 
[1] "Gene"          "RANGE"         "N_INFORMATIVE" "NumVar"       
[5] "NumPolyVar"    "Q"             "rho"           "Pvalue"       
norw suppDf: 596 
[1] "Gene"    "numVars"
length: 42 
TRUE   TRUE   TRUE 
      Gene paper_NumVars paper_NumVars_FoundinAMPPD
1      AGA            10                          4
2     ARSA             5                          2
3     ARSB            10                          5
4    ASAH1            17                          9
5  ATP13A2            18                          4
6     CLN3            17                          4
7     CLN6             7                          3
8     CLN8             4                          2
9     CTNS            12                          4
10    CTSA            11                          2
11    CTSF             9                          3
12    CTSK             5                          1
13   FUCA1            12            

In [74]:
#copy
#fileName='../LSD-All-genes/stats/LSD-genes_num_vars-all-NonSyn-CADD.txt'
#cat('outFile:', fileName,'\n')

#
if(0)
{
    cmd=str_glue('gsutil -u {BILLING_PROJECT_ID} cp -r {resDir}* {WORKSPACE_BUCKET}')
    cat('cmd:', cmd, '\n')
    shell_do(cmd)
}


if(1)
{
    #fileName='/home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat_allRaces.tab'
    fileName='/home/jupyter-user/notebooks/LSDProject/results/res_Feb1021_47LSDGenes_usingRefFlatFile/'
    fileName='/home/jupyter-user/notebooks/LSDProject/data/cmp_variants_across_cohorts.txt'
    cmd=str_glue('gsutil -u {BILLING_PROJECT_ID} cp {fileName}* {WORKSPACE_BUCKET}')
    cat('cmd:', cmd, '\n')
    shell_do(cmd)
}



    
print('finished copying')



cmd: gsutil -u larobak11 cp /home/jupyter-user/notebooks/LSDProject/data/cmp_variants_across_cohorts.txt* gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5 
[1] "Executing:  gsutil -u larobak11 cp /home/jupyter-user/notebooks/LSDProject/data/cmp_variants_across_cohorts.txt* gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5"


character(0)

[1] "finished copying"


# GBA only burden

In [40]:
#remove race column
fileName='/home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat_allRaces_PPMI.tab'
tmpDf=read.table(fileName,  header=T, sep='\t',stringsAsFactors = F, check.names=F)
cat('nrow:', nrow(tmpDf))
tmpDf$race=NULL
write.table(tmpDf, file=fileName, sep='\t', row.names = F, quote = F)

print('Done!')

nrow: 1334[1] "Done!"


In [16]:
#burden test 
#set up Dirs
#workDir='/home/jupyter-user/notebooks/LaurieRobak1/LSD-All-genes'
#dataDir='/home/jupyter-user/notebooks/LaurieRobak1/data'
#rvtestCmd='/home/jupyter-user/notebooks/LaurieRobak1/tools/executable/rvtest'
#phenoFile='/home/jupyter-user/notebooks/LaurieRobak1/data/plink_test_pheno_withPCs_rvtestFormat.tab'
dataDir='/home/jupyter-user/notebooks/LSDProject/data/'
rvtestCmd='/home/jupyter-user/notebooks/LSDProject/tools/rvtests/executable/rvtest'
#phenoFile='/home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat.tab'
#phenoFile='/home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat_allRaces_PPMI.tab'
phenoFile='/home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat_allRaces.tab'
#COV_NAMES="sex,AgeBaseLine,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10"
COV_NAMES="sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20"
#select maf cut
mafCut=0.03
#mafCut=0.01
if(mafCut==0.03)
{
  mafCutStr='maf003'  
}else if(mafCut==0.01)
{
  mafCutStr='maf001'
}
cat('mafCutStr:', mafCutStr, '\n')
#result dir
#resDir='/home/jupyter-user/notebooks/LSDProject/results/tmp/'
resDir='/home/jupyter-user/notebooks/LSDProject/results/res_Feb1021_GBAonly/'


#select the type of input based on filter
#filterType='nonSynFilt'
filterType='nonSynExonicFilt'
#filterType='CADDFilt'
cat('filterType:', filterType, '\n')
inputPrefix=str_glue('LSDGenes-WGS-AMPPD_{filterType}-VCF')
vcfFile=str_glue('{dataDir}{inputPrefix}.vcf.gz')
cat('vcfFile:', vcfFile, '\n')

#select study
#studyName='PPMI'

#gene file
#SMPD1_refFlat_hg38.txt
geneName='GBA'
#geneName='SMPD1'
#geneFile=str_glue('{dataDir}{geneName}_refFlat_hg38.txt')
geneFile=str_glue('{dataDir}refFlat_hg38.txt')
#outPrefix=str_glue('{resDir}{geneName}_{filterType}.{mafCutStr}')
outPrefix=str_glue('{resDir}{geneName}_{filterType}.{mafCutStr}')
cat('outPrefix:', outPrefix, '\n')

cmd=str_glue('{rvtestCmd} --noweb --hide-covar --inVcf {vcfFile} ',
'--pheno {phenoFile} ' ,
'--pheno-name PHENO ' ,
'--covar {phenoFile} ',
'--covar-name {COV_NAMES} ',
'--kernel skato ',
'--geneFile {geneFile} ',
'--gene {geneName} ',
'--freqUpper {mafCut} ',
'--out {outPrefix}'
)
print(cmd)
shell_do(cmd)
print('Done!!')   


mafCutStr: maf003 
filterType: nonSynExonicFilt 
vcfFile: /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_nonSynExonicFilt-VCF.vcf.gz 
outPrefix: /home/jupyter-user/notebooks/LSDProject/results/res_Feb1021_GBAonly/GBA_nonSynExonicFilt.maf003 
/home/jupyter-user/notebooks/LSDProject/tools/rvtests/executable/rvtest --noweb --hide-covar --inVcf /home/jupyter-user/notebooks/LSDProject/data/LSDGenes-WGS-AMPPD_nonSynExonicFilt-VCF.vcf.gz --pheno /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat_allRaces.tab --pheno-name PHENO --covar /home/jupyter-user/notebooks/LSDProject/data/plink_pheno_withPCs_rvtestFormat_allRaces.tab --covar-name sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20 --kernel skato --geneFile /home/jupyter-user/notebooks/LSDProject/data/refFlat_hg38.txt --gene GBA --freqUpper 0.03 --out /home/jupyter-user/notebooks/LSDProject/results/res_Feb1021_GBAonly/GBA_nonSynExonicFilt.maf003
[1] "

[1] "Thank you for using rvtests (version: 20190205, git: c86e589efef15382603300dc7f4c3394c82d69b8)"
[2] "  For documentations, refer to http://zhanxw.github.io/rvtests/"                               
[3] "  For questions and comments, plase send to Xiaowei Zhan <zhanxw@umich.edu>"                   
[4] "  For bugs and feature requests, please submit at: https://github.com/zhanxw/rvtests/issues"   
[5] ""                                                                                              
[6] "RVTESTS finished successfully"

[1] "Done!!"


In [57]:
fileName='/home/jupyter-user/notebooks/LSDProject/results/tmp/GBA.maf001.SkatO.assoc'
cmd=str_glue('gsutil -u {BILLING_PROJECT_ID} cp {fileName} {WORKSPACE_BUCKET}')
shell_do(cmd)
print('Done')


[1] "Executing:  gsutil -u larobak11 cp /home/jupyter-user/notebooks/LSDProject/results/tmp/GBA.maf001.SkatO.assoc gs://fc-secure-faef21d4-0ae2-4dfa-a025-d9292ca13bf5"


character(0)

[1] "Done"
